In [ ]:
# Main Cell 1: Load environment variables and prepare FastAPI
from dotenv import load_dotenv
load_dotenv()   # this reads your existing .env in the project root
import os, openai, json
from openai import OpenAI
# pull key from environment
# Note: you can also set OPENAI_API_KEY in your .env file
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# Quick check
assert client.api_key, "OPENAI_API_KEY not found in environment" # load key in .env
print("Loaded key, length:", len(client .api_key))
# What we want back from LLM
extract_event_schema = [
    {
        "name": "extract_event",
        "description": "Parse a scheduling request into structured event fields",
        "parameters": {
            "type": "object",
            "properties": {
                "title":     {"type": "string", "description": "A short event title"},
                "date":      {"type": "string", "description": "YYYY-MM-DD"},
                "start":     {"type": "string", "description": "HH:MM, 24-hour"},
                "end":       {"type": "string", "description": "HH:MM, 24-hour"},
                "tz":        {"type": "string", "description": "IANA timezone, e.g. America/Toronto"},
                "location":  {"type": "string", "description": "Optional location or URL"}
            },
            "required": ["title","date","start","end","tz"]
        }
    }
]
import nest_asyncio
nest_asyncio.apply()  # allow uvicorn in-notebook

from fastapi import FastAPI, Request, Response
from icalendar import Calendar, Event
from datetime import datetime
import pytz, uuid

app = FastAPI()
from pydantic import BaseModel, Field
from typing  import Optional



class ICSRequest(BaseModel):
    title:     str       = Field(..., example="Team sync")
    date:      str       = Field(..., example="2025-05-15")   # or date type
    start:     str       = Field(..., example="10:00")
    end:       str       = Field(..., example="11:00")
    tz:        str       = Field(..., example="America/Toronto")
    location:  Optional[str] = Field(None, example="Zoom Meeting")




Loaded key, length: 164


In [ ]:
# helper cell 1 get time now
from datetime import datetime

# 1. Get the current local datetime with tzinfo
local_now    = datetime.now().astimezone()
local_tzinfo = local_now.tzinfo

# 2. Derive a tz-name string for the prompt
local_tzname = getattr(local_tzinfo, "key", local_tzinfo.tzname(None))

# 3. Build ISO timestamp including offset
now_iso = local_now.strftime("%Y-%m-%dT%H:%M:%S%z")

print(f"Detected local time: {now_iso} ({local_tzname})")


Detected local time: 2025-05-19T18:08:14-0400 (东部夏令时)


In [ ]:
# helper cell 2 extract_slots
def extract_slots(nl: str, now_iso: str, tzname: str) -> dict:
    system_prompt = (
        f"You are a scheduling assistant.  The current date/time is {now_iso} "
        f"in timezone {tzname}.  "
        "When the user says 'tomorrow', 'next Friday', or similar, interpret relative to that.  "
        "Return JSON ONLY by calling the extract_event function."
    )
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system", "content": system_prompt},
            {"role":"user",   "content": nl}
        ],
        functions=extract_event_schema,
        function_call={"name":"extract_event"}
    )
    fn = resp.choices[0].message.function_call
    return json.loads(fn.arguments)


In [18]:
# Cell 2: root check
@app.get("/")
def read_root():
    return {"message": "AI Scheduler (notebook) running"}

In [ ]:
# Cell 3: test the API
from fastapi import Response

@app.post(
    "/generate-ics",
    response_description="An ICS calendar file",
    response_class=Response,
    responses={200: {"content": {"text/calendar": {}}}}
)
async def generate_ics(payload: ICSRequest):
    # now `payload` is a validated ICSRequest
    cal = Calendar()
    cal.add("prodid", "-//AIScheduler//EN")
    cal.add("version", "2.0")

    # parse and localize
    tz = pytz.timezone(payload.tz)
    start = tz.localize(datetime.fromisoformat(f"{payload.date}T{payload.start}"))
    end   = tz.localize(datetime.fromisoformat(f"{payload.date}T{payload.end}"))

    event = Event()
    event.add("uid", str(uuid.uuid4()))
    event.add("dtstamp", datetime.utcnow().replace(tzinfo=pytz.UTC))
    event.add("dtstart", start)
    event.add("dtend",   end)
    event.add("summary", payload.title)
    if payload.location:
        event.add("location", payload.location)

    cal.add_component(event)
    ics_bytes = cal.to_ical()

    return Response(
        content=ics_bytes,
        media_type="text/calendar",
        headers={"Content-Disposition": f"attachment; filename={payload.title}.ics"}
    )


In [ ]:
# Cell 4: background server
import threading, uvicorn

def _run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

thread = threading.Thread(target=_run_server, daemon=True)
thread.start()

print("Server started in background on http://127.0.0.1:8000")


Server started in background on http://127.0.0.1:8000


INFO:     Started server process [27496]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [ ]:
# Cell 3: chat loop
import sys, json, requests
from datetime import datetime

while True:
    # 1) Read raw input
    user_text = input("\nYou(exit/quit to exit): ").strip()

    # 2) Check for exit BEFORE calling the LLM
    if user_text.lower() in ("exit", "quit"):
        print("Goodbye!")
        break

    # 3) Build the “now” context
    local_now    = datetime.now().astimezone()
    now_iso      = local_now.strftime("%Y-%m-%dT%H:%M:%S%z")
    tzinfo       = local_now.tzinfo
    tzname       = getattr(tzinfo, "key", tzinfo.tzname(None))

    # 4) Extract slots
    try:
        ev = extract_slots(user_text, now_iso, tzname)
    except Exception as e:
        print("Extraction error:", e)
        continue

    # 5) Debug print (optional)
    print("Debug extracted slots:", json.dumps(ev, indent=2), flush=True)

    # 6) Human‐readable summary
    print("\nEvent to add:", flush=True)
    print(f" • Title:    {ev.get('title')}", flush=True)
    print(f" • When:     {ev.get('date')}  {ev.get('start')}–{ev.get('end')} ({tzname})", flush=True)
    if ev.get("location"):
        print(f" • Location: {ev.get('location')}", flush=True)

    # 7) Confirm prompt
    confirm = input("\nConfirm and generate .ics? (y/n): ").strip().lower()
    if confirm != "y":
        print("Okay, let's try again.", flush=True)
        continue

    # 8) Hit API and save the file
    resp = requests.post("http://127.0.0.1:8000/generate-ics", json=ev)
    if resp.status_code == 200:
        fname = ev["title"].replace(" ", "_") + ".ics"
        with open(fname, "wb") as f:
            f.write(resp.content)
        print(f"Saved: {fname}", flush=True)
    else:
        print("Failed to generate ICS:", resp.status_code, resp.text, flush=True)


🤖 Debug – extracted slots: {
  "title": "Pick up Aya at YVR Airport",
  "date": "2025-05-20",
  "start": "15:30",
  "end": "16:00",
  "tz": "America/Toronto",
  "location": "YVR Airport"
}

Event to add:
 • Title:    Pick up Aya at YVR Airport
 • When:     2025-05-20  15:30–16:00 (东部夏令时)
 • Location: YVR Airport
✅ Saved: Pick_up_Aya_at_YVR_Airport.ics
🤖 Debug – extracted slots: {
  "title": "Unnamed Event",
  "date": "2025-05-20",
  "start": "18:09",
  "end": "19:09",
  "tz": "America/Toronto"
}

Event to add:
 • Title:    Unnamed Event
 • When:     2025-05-20  18:09–19:09 (东部夏令时)
Okay, let's try again.
Goodbye! 👋
